In [8]:
import pandas as pd
df_final_demo = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_demo.txt")
df_final_experiment_clients = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_experiment_clients.txt")
df_final_web_data_pt_1 = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_web_data_pt_1.txt")
df_final_web_data_pt_2 = pd.read_csv(r"/Users/muayadhilamia/Desktop/Ironhack/Week-5/week5_6_project/data/raw/df_final_web_data_pt_2.txt")


In [15]:
# Concating df_final_web p1 and p2 and merging them with df_final_experiment_clients
df_final_web_data = pd.concat([df_final_web_data_pt_1, df_final_web_data_pt_2], ignore_index=True)
df_web_experiment = pd.merge(
    df_final_web_data,
    df_final_experiment_clients,
    on="client_id",
    how="inner"
)

In [25]:
df_web_experiment["process_step"].unique()

array(['step_3', 'step_2', 'step_1', 'start', 'confirm'], dtype=object)

In [28]:
# cleaning and droping duplicated and null values in Variation 
df_web_experiment = df_web_experiment.drop_duplicates()
df_web_experiment = df_web_experiment.dropna(subset=['Variation'])

# replacing steps by numbers 
df_web_experiment["process_step"] = (df_web_experiment["process_step"].replace({
    "start": 1,
    "step_1": 2,
    "step_2": 3,
    "step_3": 4,
    "confirm": 5
}).astype(int))

In [30]:
df_web_experiment.to_csv("df_web_experiment_cleand_steps")

----------------------

In [11]:
# SettingWithCopyWarning fixing
df_web_experiment = df_web_experiment.copy()

# Making num_visit col (client_id)
client_visits = (
    df_web_experiment
    .groupby('client_id')['visit_id']
    .nunique()
    .reset_index(name='num_visits')
)

df_web_experiment.loc[:, 'num_visits'] = (
    df_web_experiment
    .groupby('client_id')['visit_id']
    .transform('nunique')
)

# Making num_steps (visit_id)
df_web_experiment.loc[:, 'num_steps'] = (
    df_web_experiment
    .groupby('visit_id')['process_step']
    .transform('count')
)

# Fixing date_time col 
df_web_experiment.loc[:, 'date_time'] = pd.to_datetime(df_web_experiment['date_time'], errors='coerce')

# Making last_step col 
df_web_experiment.loc[:, 'last_step'] = (
    df_web_experiment
    .sort_values(['visit_id', 'date_time'])
    .groupby('visit_id')['process_step']
    .transform('last')
)

# Making visit duration col (minutes:seconds format)
df_web_experiment.loc[:, 'visit_duration'] = (
    df_web_experiment
    .groupby('visit_id')['date_time']
    .transform(
        lambda x: str(
            f"{int((x.max() - x.min()).total_seconds() // 60):02d}:{int((x.max() - x.min()).total_seconds() % 60):02d}"
        ) if pd.notnull(x.max()) and pd.notnull(x.min()) else None
    )
).astype('string')

# filling missing in variation to unknown 
df_web_experiment['Variation'] = df_web_experiment['Variation'].fillna('unknown')

----------------

In [12]:
# df with the unique visit id with Variation
df_unique_visits = df_web_experiment.drop_duplicates(subset='visit_id', keep='first')


In [13]:
df_web_experiment

,client_id,visitor_id,visit_id,process_step,date_time,Variation,num_visits,num_steps,last_step,visit_duration
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,2,8,step_3,10:45
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,2,8,step_3,10:45
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,2,8,step_3,10:45
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,2,8,step_3,10:45
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,2,8,step_3,10:45
...,...,...,...,...,...,...,...,...,...,...
449826,9895983,473024645_56027518531,498981662_93503779869_272484,step_3,2017-06-15 19:52:09,unknown,1,6,confirm,11:04
449827,9895983,473024645_56027518531,498981662_93503779869_272484,step_2,2017-06-15 19:50:37,unknown,1,6,confirm,11:04
449828,9895983,473024645_56027518531,498981662_93503779869_272484,step_1,2017-06-15 19:50:05,unknown,1,6,confirm,11:04
449829,9895983,473024645_56027518531,498981662_93503779869_272484,start,2017-06-15 19:50:00,unknown,1,6,confirm,11:04
